# Time series
75033710 rows

13713 unique time series taken in 12603 stations from 17-10-2000 to 22-03-2018

In [1]:
import pyarrow.feather as feather
import pandas as pd
import numpy as np
from scipy.io import netcdf
import matplotlib.pyplot as plt 

# temperature: as predictor

Daily means of hourly grids of air temperature for whole Germany (project TRY Advancement)

there are 215 NC files, each file represents a month of dailly temperature from 01-1995 tp 11-2012



In [ ]:
# extract relevant temp to T.S from an nc file 

In [2]:
# each nc file has temperature across whole germany
dir_01_2005 = "dataset/project_try_dwd/data/"+'daily_Project_TRY_air_temperature_mean__TT_200511_daymean.nc'
temp_01_2005 = netcdf.NetCDFFile(dir_01_2005,'r')

In [3]:
temp_01_2005.variables

OrderedDict([('lon', <scipy.io.netcdf.netcdf_variable at 0x7faf89466190>),
             ('lat', <scipy.io.netcdf.netcdf_variable at 0x7faf89cdac70>),
             ('time', <scipy.io.netcdf.netcdf_variable at 0x7faf89cd54c0>),
             ('time_bnds',
              <scipy.io.netcdf.netcdf_variable at 0x7faf58fa8520>),
             ('datum', <scipy.io.netcdf.netcdf_variable at 0x7faf58fa8910>),
             ('temperature',
              <scipy.io.netcdf.netcdf_variable at 0x7faf58fa8a00>)])

In [58]:
# long and lat where temp was measured 
x_long = temp_01_2005.variables["lon"][:].flatten()
y_lat = temp_01_2005.variables["lat"][:].flatten()


In [61]:
x_long.shape

(675360,)

In [5]:
938*720

675360

In [56]:
meta_df = pd.read_excel("dataset/metadata_timeseries_LongLat.xlsx")
meta_df.head()

,proj_id,name,x_long,y_lat,x_utm32,y_utm32,aquifer_id,filteroberkante,filterunterkante,filter_length,filter_depth_middle,surface_elevation,watertable_pressure,well_depth,aquifer_level
0,BB_25470023,Ottenhagen OP,13.643609,53.416804,808527.0,5928687.0,weitgehend bedeckter GWL,NaN,NaN,NaN,NaN,"100,33",gespannt,"62,53",NaN
1,BB_25470024,Ottenhagen UP,13.643653,53.416793,808530.0,5928686.0,tieferer GWL in tiefern quart. und tert. Schic...,NaN,NaN,NaN,NaN,"100,38",NaN,"45,68",NaN
2,BB_25480025,Neumannshof,13.826532,53.452024,820407.0,5933409.0,weitgehend bedeckter GWL,NaN,NaN,NaN,NaN,"58,94",gespannt,"10,84",NaN
3,BB_25480026,Amalienhof,13.677407,53.457078,810477.0,5933310.0,weitgehend bedeckter GWL,NaN,NaN,NaN,NaN,"97,67",gespannt,"62,67",NaN
4,BB_25481088,"Milow, Beschaffenheitsmesst.OP",13.821475,53.475857,819892.0,5936035.0,weitgehend unbedeckter GWL,NaN,NaN,NaN,NaN,"57,6",gespannt,"19,71",NaN


In [57]:
# create df to store the temp T.S for each station

station_temp_df =meta_df[["proj_id","x_long","y_lat"]]
station_temp_df.head()

,proj_id,x_long,y_lat
0,BB_25470023,13.643609,53.416804
1,BB_25470024,13.643653,53.416793
2,BB_25480025,13.826532,53.452024
3,BB_25480026,13.677407,53.457078
4,BB_25481088,13.821475,53.475857


In [62]:
station_temp_df.shape

(13492, 3)

In [69]:
x_long[np.argmin(np.absolute(x_long-13.643609))]

13.6436

In [ ]:
# extract the indices of the temp - matching the locations between weather stations ans bgr stations

def temp_idx(df):
    # compute the temp_index according the closest long_lat
    long = df["x_long"]
    lat  = df["y_lat"]
    # compute the closest temp_long to the station- all occurances
    a = np.absolute(x_long - long)
    # compute the closest temp_lat to the station- all occurances
    b = np.absolute(y_lat - lat)

    if np.min(a) <= np.min(b):
        min_long = np.argwhere(a == np.min(a))
        idx = [int(i) for i in min_long if b[i] == np.min(b[min_long])][0]

    else:
        min_lat = np.argwhere(b == np.min(b))
        idx = [int(i) for i in min_lat if a[i] == np.min(a[min_lat])][0]
    
    return idx



In [ ]:
# the indicies computed once and stored 
station_temp_df["idx"] = station_temp_df.apply(temp_idx,axis=1) 
# station_temp_df.head()
#station_temp_df = station_temp_df.rename(columns={"Temp_20050103":"idx"})
# save in temporary dataset file for later use
#station_temp_df.to_csv("dataset/temporary/station_temp_df.csv", index=False)

In [11]:
# create df for the lat/long of bgr station and corresponding to ind od weather station 
temp_df = pd.read_csv("dataset/temporary/station_temp_df.csv")
temp_df.head()

,proj_id,x_long,y_lat,idx
0,BB_25470023,13.643609,53.416804,515351
1,BB_25470024,13.643653,53.416793,515351
2,BB_25480025,13.826532,53.452024,522658
3,BB_25480026,13.677407,53.457078,584481
4,BB_25481088,13.821475,53.475857,557852


In [14]:
# pathes to all nc files
from os import walk
folder_path =  "dataset/project_try_dwd/data/"
files= []
for (dirpath, dirnames, filenames) in walk(folder_path):
    files.extend(filenames)
    break
files[:2]

['daily_Project_TRY_air_temperature_mean__TT_200912_daymean.nc',
 'daily_Project_TRY_air_temperature_mean__TT_199602_daymean.nc']

In [ ]:
# extract temps in each day of each nc file at the corresponding lat/longfor all months
for file in files:
    month = folder_path + file
    nc_file = netcdf.NetCDFFile(month,'r')
    days = nc_file.variables["datum"][:]
    days = [str(x)[:-4] for x in days]
    for d, day in enumerate(days):
        #if day in unique_dates:
        temp_df[str(day)] = nc_file.variables["temperature"][d,:,:].flatten()[temp_df["idx"]]


In [73]:
a =temp_01_2005.variables["temperature"][5,:,:].flatten()[temp_df["idx"]]

In [75]:
len(a[a==9999])

1521

In [49]:
b =temp_01_2005.variables["temperature"][20,:,:].flatten()[]
b

array([  14,   14, 9999, ..., 9999, 9999,   -3], dtype=int16)

In [50]:
long = df["x_long"]
lat  = df["y_lat"]
# compute the closest temp_long to the station- all occurances
a = np.absolute(x_long - long)
# compute the closest temp_lat to the station- all occurances
b = np.absolute(y_lat - lat)

if np.min(a) <= np.min(b):
    min_long = np.argwhere(a == np.min(a))
    idx = [int(i) for i in min_long if b[i] == np.min(b[min_long])][0]

else:
    min_lat = np.argwhere(b == np.min(b))
    idx = [int(i) for i in min_lat if a[i] == np.min(a[min_lat])][0]


(1521,)

In [ ]:
dirc = "dataset/project_try_dwd/data/"+files[0]
dirc

In [ ]:
plt.figure(figsize=(50,15))
t_s_1.plot(color='blue', grid=True)
plt.legend(loc='upper left')
plt.xlabel('t s temp_BB_25470023 ')
temp_BB_254700230.plot(color='red', grid=True, secondary_y=True)
plt.legend(loc='upper right')
plt.xlabel('temp temp_BB_25470023 ')
plt.show()